In [39]:
import pandas as pd

In [40]:
def find_in_cols(df, search_term, search_cols):
    if not search_cols:
        return df.iloc[0:0]  # Return empty DataFrame if no columns specified

    mask = pd.Series(False, index=df.index)
    for col in search_cols:
        if col in df.columns:
            mask |= df[col].astype(str).str.contains(search_term, case=False, na=False)
    return df[mask]


In [41]:

mal_df_raw = pd.read_csv(r'data/myanimelist_recommender_ready.csv')
mal_df_raw.head()

,mal_id,title,title_english,title_japanese,type,episodes,status,score,rank,popularity,members,favorites,synopsis,genres,themes,demographics,studios,year,url
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,TV,26.0,Finished Airing,8.75,46.0,42,1975882,86621,"Crime is timeless. By the year 2071, humanity ...","Action, Award Winning, Sci-Fi","Adult Cast, Space",NaN,Sunrise,1998.0,https://myanimelist.net/anime/1/Cowboy_Bebop
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,Movie,1.0,Finished Airing,8.38,215.0,642,398122,1711,"Another day, another bounty—such is the life o...","Action, Sci-Fi","Adult Cast, Space",NaN,Bones,NaN,https://myanimelist.net/anime/5/Cowboy_Bebop__...
2,6,Trigun,Trigun,トライガン,TV,26.0,Finished Airing,8.22,378.0,260,803358,16863,"Vash the Stampede is the man with a $$60,000,0...","Action, Adventure, Sci-Fi",Adult Cast,Shounen,Madhouse,1998.0,https://myanimelist.net/anime/6/Trigun
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26.0,Finished Airing,7.23,3260.0,1950,123424,674,"Though hidden away from the general public, Wi...","Action, Drama, Mystery, Supernatural",Detective,NaN,Sunrise,2002.0,https://myanimelist.net/anime/7/Witch_Hunter_R...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,TV,52.0,Finished Airing,6.92,4819.0,5655,16308,16,It is the dark century and the people are suff...,"Action, Adventure, Fantasy",NaN,Shounen,Toei Animation,2004.0,https://myanimelist.net/anime/8/Bouken_Ou_Beet


In [42]:
mal_df_raw.shape

(28707, 19)

In [43]:
mal_df_raw['status'].unique()

array(['Finished Airing', 'Currently Airing', 'Not yet aired'],
      dtype=object)

In [44]:
# Sort theo thời gian 
# status: 'Finished Airing' < 'Currently Airing' < 'Not Yet Aired', và year tăng dần
status_order = ['Finished Airing', 'Currently Airing', 'Not Yet Aired']
mal_df_raw['status'] = pd.Categorical(mal_df_raw['status'], categories=status_order, ordered=True)

mal_df_sorted = mal_df_raw.sort_values(by=['status', 'year'])


In [45]:
mal_df_sorted.shape

(28707, 19)

In [46]:
mal_df_sorted[mal_df_sorted['year'].isna()].shape

(22515, 19)

In [47]:
find_in_cols(mal_df_sorted, 'Project sekai', ['title',	'title_english',	'title_japanese'])

,mal_id,title,title_english,title_japanese,type,episodes,status,score,rank,popularity,members,favorites,synopsis,genres,themes,demographics,studios,year,url
27049,59419,Project Sekai Movie: Kowareta Sekai to Utaenai...,Colorful Stage! The Movie: A Miku Who Can't Sing,劇場版プロジェクトセカイ 壊れたセカイと歌えないミク,Movie,1.0,Finished Airing,7.67,1386.0,4966,22783,314,"At a CD store, Ichika Hoshino hears a song by ...",NaN,Music,NaN,P.A. Works,NaN,https://myanimelist.net/anime/59419/Project_Se...


In [48]:
import requests
import pandas as pd
import time
import os
import ast
from collections import Counter
from tqdm import tqdm
from IPython.display import clear_output

def get_info(anime_id):
    url = f"https://api.jikan.moe/v4/anime/{anime_id}/full"
    response = requests.get(url)
    time.sleep(0.5)
    if response.status_code == 200:
        data = response.json()
        return data['data'] if data.get('data') else '{}'
    return response

def get_reviews(anime_id):
    url = f"https://api.jikan.moe/v4/anime/{anime_id}/reviews"
    response = requests.get(url)
    time.sleep(0.5)
    if response.status_code == 200:
        data = response.json()
        return data['data'] if data.get('data') else '[]'
    return response

def count_review_tags(reviews):
    if not reviews or reviews == '[]':
        return None
    tag_counter = Counter()
    for review in reviews:
        tags = review.get('tags', [])
        tag_counter.update(tags)
    return dict(tag_counter)

def safe_eval(val):
    try:
        return ast.literal_eval(val)
    except Exception:
        return None

def enrich_anime_dataframe(df, save_path='anime_enriched.csv', checkpoint_interval=50):
    print(f"🔍 Starting enrichment for {len(df)} anime entries...\n")

    if os.path.exists(save_path):
        enriched_df = pd.read_csv(save_path, converters={"review_tags_count": safe_eval})
        completed_ids = set(enriched_df['mal_id'])
        print(f"🔁 Resuming from checkpoint: {len(completed_ids)} already processed.")
    else:
        enriched_df = pd.DataFrame(columns=df.columns.tolist() + ['aired_date', 'review_tags_count'])
        completed_ids = set()

    row_count = 0
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        mal_id = row['mal_id']
        mal_title = row['title']
        mal_jp_title = row['title_japanese']

        if mal_id in completed_ids:
            continue

        row_count += 1
        if row_count % 5 == 0:
            clear_output(wait=True)

        print(f"\n[{i+1}/{len(df)}] MAL ID: {mal_id} - Title: {mal_title} / {mal_jp_title}")

        # --- Retry get_info() until success ---
        error_count = 0
        while True:
            try:
                info = get_info(mal_id)
            except Exception as e:
                print(f"❌ get_info() failed for MAL ID {mal_id} — Exception: {e}")
                print("⏳ Waiting 30s and retrying...")
                time.sleep(30)
                error_count += 1
                if error_count > 5:
                    print(f"🚨 Too many errors for MAL ID {mal_id}. Skipping...")
                    info = '{}'
                    break
                continue

            if isinstance(info, requests.models.Response):
                print(f"❌ get_info() failed for MAL ID {mal_id} — status {info.status_code}")
                print("🔎 Response:", info.text)
                print("⏳ Waiting 30s and retrying...")
                time.sleep(30)
                error_count += 1
                if error_count > 5:
                    print(f"🚨 Too many errors for MAL ID {mal_id}. Skipping...")
                    info = '{}'
                    break
            else:
                break

        # Even if info is empty '{}', append with null aired_date
        aired_date = info.get('aired', {}).get('from') if info != '{}' else None

        print(f"📅 Aired date: {aired_date if aired_date else 'N/A'}")

        # --- Retry get_reviews() until success ---
        error_count = 0
        while True:
            try:
                reviews = get_reviews(mal_id)
            except Exception as e:
                print(f"❌ get_reviews() failed for MAL ID {mal_id} — Exception: {e}")
                print("⏳ Waiting 30s and retrying...")
                time.sleep(30)
                error_count += 1
                if error_count > 5:
                    print(f"🚨 Too many errors for MAL ID {mal_id}. Skipping...")
                    reviews = '[]'
                    break
                continue
            if isinstance(reviews, requests.models.Response):
                print(f"❌ get_reviews() failed for MAL ID {mal_id} — status {reviews.status_code}")
                print("🔎 Response:", reviews.text)
                print("⏳ Waiting 30s and retrying...")
                time.sleep(30)
                error_count += 1
                if error_count > 5:
                    print(f"🚨 Too many errors for MAL ID {mal_id}. Skipping...")
                    reviews = '[]'
                    break
            else:
                break

        tags = count_review_tags(reviews)

        # Append result (even if values are None)
        enriched_row = row.copy()
        enriched_row['aired_date'] = aired_date
        enriched_row['review_tags_count'] = tags
        enriched_df = pd.concat([enriched_df, pd.DataFrame([enriched_row])], ignore_index=True)

        # Checkpoint save
        if len(enriched_df) % checkpoint_interval == 0:
            enriched_df.to_csv(save_path, index=False)
            print(f"💾 Checkpoint saved: {save_path} — {len(enriched_df)} rows")

    # Final save
    enriched_df.to_csv(save_path, index=False)
    print(f"\n✅ Done! Final saved CSV: {save_path}")
    return enriched_df


In [49]:
def fix_missing_aired_dates(df, save_path='anime_enriched_fixed.csv'):
    import time
    from tqdm import tqdm

    df = df.copy()  # So original is unchanged
    needs_update = df[(df['aired_date'].isna()) & 
                      (df['status'].isin(['Finished Airing', 'Currently Airing']))]

    print(f"🔍 Found {len(needs_update)} entries with missing aired_date and valid status.\n")

    for i, row in tqdm(needs_update.iterrows(), total=len(needs_update), desc="Fixing aired_date"):
        mal_id = row['mal_id']
        title = row.get('title', 'Unknown')

        print(f"\n⏳ Rechecking MAL ID: {mal_id} - {title}")

        error_count = 0
        while True:
            try:
                info = get_info(mal_id)
            except Exception as e:
                print(f"❌ Exception for MAL ID {mal_id}: {e}")
                time.sleep(30)
                error_count += 1
                if error_count > 5:
                    print("🚨 Too many errors. Skipping.")
                    info = '{}'
                    break
                continue

            if isinstance(info, requests.models.Response):
                print(f"❌ get_info() failed — status {info.status_code}")
                print("🔎 Response:", info.text)
                time.sleep(30)
                error_count += 1
                if error_count > 5:
                    print("🚨 Too many errors. Skipping.")
                    info = '{}'
                    break
            else:
                break

        new_date = info.get('aired', {}).get('from') if info != '{}' else None
        print(f"📅 Updated aired_date: {new_date if new_date else 'N/A'}")

        df.loc[df['mal_id'] == mal_id, 'aired_date'] = new_date

    df.to_csv(save_path, index=False)
    print(f"\n✅ Fixed aired_date saved to: {save_path}")
    return df


In [50]:
# save_path='anime_enriched.csv'
# if os.path.exists(save_path):
#     check_df = pd.read_csv(save_path, converters={"review_tags_count": safe_eval})
#     completed_ids = set(check_df['mal_id'])
#     print(f"🔁 Resuming from checkpoint: {len(completed_ids)} already processed.")

In [51]:
#mal_df_raw = enrich_anime_dataframe(mal_df_raw)


In [52]:
mal_df_raw= pd.read_csv(r'data/myanimelist_recommender_ready.csv')
mal_df_enriched = pd.read_csv('anime_enriched.csv', converters={"review_tags_count": safe_eval})

In [ ]:
#mal_df_enriched_fixed = fix_missing_aired_dates(mal_df_enriched)

In [59]:
mal_df_enriched.head(5)

,mal_id,title,title_english,title_japanese,type,episodes,status,score,rank,popularity,...,favorites,synopsis,genres,themes,demographics,studios,year,url,aired_date,review_tags_count
0,14211,Instant History,NaN,インスタントヒストリー,TV,312.0,Finished Airing,6.14,9314.0,14301,...,11,Each three-minute short features characters le...,NaN,Historical,NaN,NaN,1961.0,https://myanimelist.net/anime/14211/Instant_Hi...,1961-05-01T00:00:00+00:00,None
1,14213,Otogi Manga Calendar,NaN,おとぎマンガカレンダー,TV,312.0,Finished Airing,6.04,9868.0,14864,...,8,A continuation of Instant History.\n\n(Source:...,NaN,Historical,NaN,NaN,1962.0,https://myanimelist.net/anime/14213/Otogi_Mang...,1962-06-25T00:00:00+00:00,None
2,2686,Tetsujin 28-gou,Gigantor,鉄人２８号,TV,96.0,Finished Airing,6.38,7902.0,9523,...,13,Dr.Haneda was developing experimental giant ro...,"Adventure, Sci-Fi",Mecha,Shounen,Eiken,1963.0,https://myanimelist.net/anime/2686/Tetsujin_28...,1963-10-20T00:00:00+00:00,"{'Recommended': 1, 'Not Recommended': 1}"
3,2747,Tetsuwan Atom,Astro Boy,鉄腕アトム,TV,193.0,Finished Airing,7.08,4056.0,5938,...,59,Devastated over losing his only child in a car...,"Action, Adventure, Drama, Sci-Fi",NaN,Shounen,Mushi Production,1963.0,https://myanimelist.net/anime/2747/Tetsuwan_Atom,1963-01-01T00:00:00+00:00,{'Mixed Feelings': 1}
4,5052,Eightman,8 Man,エイトマン,TV,56.0,Finished Airing,5.89,10669.0,11286,...,8,A private investigator working a routine case ...,"Action, Drama, Sci-Fi",Mecha,NaN,TCJ,1963.0,https://myanimelist.net/anime/5052/Eightman,1963-11-07T00:00:00+00:00,{'Recommended': 1}


In [56]:
mal_df_raw_cleaned  = mal_df_raw.drop_duplicates(subset='mal_id', keep='first').reset_index(drop=True)

In [57]:
mal_df_raw_cleaned

,mal_id,title,title_english,title_japanese,type,episodes,status,score,rank,popularity,members,favorites,synopsis,genres,themes,demographics,studios,year,url
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,TV,26.0,Finished Airing,8.75,46.0,42,1975882,86621,"Crime is timeless. By the year 2071, humanity ...","Action, Award Winning, Sci-Fi","Adult Cast, Space",NaN,Sunrise,1998.0,https://myanimelist.net/anime/1/Cowboy_Bebop
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,Movie,1.0,Finished Airing,8.38,215.0,642,398122,1711,"Another day, another bounty—such is the life o...","Action, Sci-Fi","Adult Cast, Space",NaN,Bones,NaN,https://myanimelist.net/anime/5/Cowboy_Bebop__...
2,6,Trigun,Trigun,トライガン,TV,26.0,Finished Airing,8.22,378.0,260,803358,16863,"Vash the Stampede is the man with a $$60,000,0...","Action, Adventure, Sci-Fi",Adult Cast,Shounen,Madhouse,1998.0,https://myanimelist.net/anime/6/Trigun
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26.0,Finished Airing,7.23,3260.0,1950,123424,674,"Though hidden away from the general public, Wi...","Action, Drama, Mystery, Supernatural",Detective,NaN,Sunrise,2002.0,https://myanimelist.net/anime/7/Witch_Hunter_R...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,TV,52.0,Finished Airing,6.92,4819.0,5655,16308,16,It is the dark century and the people are suff...,"Action, Adventure, Fantasy",NaN,Shounen,Toei Animation,2004.0,https://myanimelist.net/anime/8/Bouken_Ou_Beet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28486,61848,Neko no Komoriuta,NaN,ねこの子もりうた,Music,1.0,Finished Airing,NaN,NaN,28490,18,0,Music video for the song Neko no Komoriuta by ...,NaN,Music,Kids,NaN,NaN,https://myanimelist.net/anime/61848/Neko_no_Ko...
28487,61849,Hirara Koi Kochou,NaN,ひらら恋胡蝶,Music,1.0,Finished Airing,NaN,NaN,28489,22,0,Music video for the song Hirara Koi Kochou by ...,NaN,Music,Kids,NaN,NaN,https://myanimelist.net/anime/61849/Hirara_Koi...
28488,61850,Kawaita Gloss,NaN,かわいたグロス,Music,1.0,Finished Airing,NaN,NaN,28486,27,0,Music video for the song Kawaita Gloss by Aiko...,NaN,Music,NaN,NaN,NaN,https://myanimelist.net/anime/61850/Kawaita_Gloss
28489,61851,Isekai Quartet 3,Isekai Quartet 3,異世界かるてっと3,TV,NaN,Not yet aired,NaN,NaN,8687,5660,12,Third season of Isekai Quartet.,"Comedy, Fantasy","Isekai, Parody",NaN,Studio PuYUKAI,NaN,https://myanimelist.net/anime/61851/Isekai_Qua...


In [60]:
# Step 1: Merge aired_date and review_tags_count into raw_cleaned
mal_df_merged = mal_df_raw_cleaned.merge(
    mal_df_enriched[['mal_id', 'aired_date', 'review_tags_count']],
    on='mal_id',
    how='left'
)

# Step 2: Extract year from aired_date
mal_df_merged['year'] = pd.to_datetime(mal_df_merged['aired_date'], errors='coerce').dt.year

# Step 3: Get all unique review tag keys
all_tags = set()
for d in mal_df_merged['review_tags_count'].dropna():
    all_tags.update(d.keys())

# Normalize column names: remove spaces
tag_columns = {tag: tag.replace(" ", "") for tag in all_tags}

# Step 4: Add new columns for each tag
for tag, clean_col in tag_columns.items():
    mal_df_merged[clean_col] = mal_df_merged['review_tags_count'].apply(
        lambda x: x.get(tag, 0) if isinstance(x, dict) else 0
    )

# Done
mal_df_raw_enriched = mal_df_merged.copy()

In [62]:
mal_df_raw_enriched.to_csv('myanimelist_recommender_ready_enriched.csv', index=False)